# 물체 탐지(Object detection)


# SSD로 물체 탐지


In [2]:
# keras 변환을 위한 패키지 설치
!pip install numpy>=1.19.0 tensorflow==1.14 keras==2.4.2 h5py==2.8.0 
#hdf5==1.10.2
!pip list

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.14.0 which is incompatible.
Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                   

## Keras 코드 다운로드

In [ ]:
# 프로젝트 다운로드
%cd /content
!git clone https://github.com/pierluigiferrari/ssd_keras
  
# 다운로드된 디렉토리로 이동
%cd ssd_keras

# 내용 확인
!pwd
!ls -al  

/content
Cloning into 'ssd_keras'...
remote: Enumerating objects: 1608, done.
remote: Total 1608 (delta 0), reused 0 (delta 0), pack-reused 1608
Receiving objects: 100% (1608/1608), 19.71 MiB | 39.73 MiB/s, done.
Resolving deltas: 100% (1056/1056), done.
/content/ssd_keras
/content/ssd_keras
total 4184
drwxr-xr-x 14 root root    4096 Dec 29 02:21 .
drwxr-xr-x  1 root root    4096 Dec 29 02:21 ..
drwxr-xr-x  2 root root    4096 Dec 29 02:21 bounding_box_utils
-rwxr-xr-x  1 root root    2882 Dec 29 02:21 CONTRIBUTING.md
drwxr-xr-x  2 root root    4096 Dec 29 02:21 data_generator
drwxr-xr-x  2 root root    4096 Dec 29 02:21 eval_utils
drwxr-xr-x  2 root root    4096 Dec 29 02:21 examples
drwxr-xr-x  8 root root    4096 Dec 29 02:21 .git
-rwxr-xr-x  1 root root      33 Dec 29 02:21 .gitattributes
drwxr-xr-x  2 root root    4096 Dec 29 02:21 .github
-rwxr-xr-x  1 root root    1190 Dec 29 02:21 .gitignore
-rw-r--r--  1 root root       0 Dec 29 02:21 __init__.py
-rwxr-xr-x  1 root root    247

## 모델 정의와 모델 로딩


이미 학습된 SSD 모델 파일 VGG_coco_SSD_300x300_iter_400000.h5은

다음 페이지의 'COCO[1]: SSD300*'에 걸린 링크로 다운 받는다.

https://github.com/pierluigiferrari/ssd_keras#download-the-original-trained-model-weights

files.upload()로 업로드하면 1시간 이상 시간이 걸린다.


대신 dropbox에 임시로 올려둔 파일에서 다운로드 한다. wget으로 다운로드 받는다.




In [ ]:
%cd /content/ssd_keras

!wget -O VGG_coco_SSD_300x300.h5 https://www.dropbox.com/s/z7sf5c8wx39fo9n/VGG_coco_SSD_300x300.h5?dl=1
  
!ls -al

/content/ssd_keras
--2020-12-29 00:27:25--  https://www.dropbox.com/s/z7sf5c8wx39fo9n/VGG_coco_SSD_300x300.h5?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/z7sf5c8wx39fo9n/VGG_coco_SSD_300x300.h5 [following]
--2020-12-29 00:27:25--  https://www.dropbox.com/s/dl/z7sf5c8wx39fo9n/VGG_coco_SSD_300x300.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1924260748e0d4ba26f06cc322.dl.dropboxusercontent.com/cd/0/get/BF_-oRNN8bdgIO9QvlemvAVpIqksUv3mQjajuarY-BPx-ggUGTPPB1Gx-dTRjIUCD5Jgc5HJaXoL9wF1P9FdJwQ7datAdRVeQnMmHj05vfNRrw/file?dl=1# [following]
--2020-12-29 00:27:26--  https://uc1924260748e0d4ba26f06cc322.dl.dropboxusercontent.com/cd/0/get/BF_-oRNN8bdgIO9QvlemvAVpIqksUv3mQjajuarY-BPx-ggUGTPPB1Gx-dTRjIUCD5Jgc5HJaXoL9w

In [ ]:
# local file upload시(VGG_coco_SSD_300x300.h5)

%cd /content/ssd_keras

from google.colab import files

# 파일이름 VGG_coco_SSD_300x300.h5
uploaded = files.upload()

!ls -al 

NameError: ignored

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [ ]:
# Set the image size.
img_height = 300
img_width = 300

In [ ]:
# 1: Build the Keras model

# K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
#weights_path = 'path/to/trained/weights/VGG_VOC0712_SSD_300x300_iter_120000.h5'
weights_path = 'VGG_coco_SSD_300x300.h5'


model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

## 실행

아래의 코드는 https://github.com/pierluigiferrari/ssd_keras/blob/master/ssd300_inference.ipynb 의 코드를 그대로 카피해 왔다.

In [ ]:
orig_images = [] # Store the images here.
input_images = [] # Store resized versions of the images here.

# We'll only load one image in this example.
img_path = 'examples/fish_bike.jpg'

orig_images.append(imread(img_path))
img = image.load_img(img_path, target_size=(img_height, img_width))
img = image.img_to_array(img) 
input_images.append(img)
input_images = np.array(input_images)

In [ ]:
# 모델 호출
y_pred = model.predict(input_images)

In [ ]:
# 특정값 이상만 필터링
confidence_threshold = 0.5

y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_thresh[0])

Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[[ 15.     0.98 117.57  -3.39 215.29 164.84]]


In [ ]:
# Display the image and draw the predicted boxes onto it.

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

plt.figure(figsize=(20,12))
plt.imshow(orig_images[0])

current_axis = plt.gca()

for box in y_pred_thresh[0]:
    # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
    xmin = box[2] * orig_images[0].shape[1] / img_width
    ymin = box[3] * orig_images[0].shape[0] / img_height
    xmax = box[4] * orig_images[0].shape[1] / img_width
    ymax = box[5] * orig_images[0].shape[0] / img_height
    color = colors[int(box[0])]
    label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})
    

NameError: ignored

<Figure size 1440x864 with 0 Axes>